In [1]:
from spatialvla.datasets import RLDSBatchTransform, RLDSDataset
from spatialvla.mobilevlm.model.mobilevlm import load_pretrained_vlm_for_vla, load_vla
from scripts.spatialvla_config import ModelArguments, TrainingArguments
import transformers
from spatialvla.datasets.rlds.utils.data_utils import PaddedCollatorForActionPrediction
from torch.utils.data import DataLoader
from spatialvla.mobilevlm.action_tokenizer import ActionTokenizer
import sys
import torch
import argparse
from PIL import Image
from pathlib import Path
import numpy as np

from spatialvla.mobilevlm.model.mobilevlm import load_vla, load_pretrained_model
from spatialvla.mobilevlm.conversation import conv_templates, SeparatorStyle
from spatialvla.mobilevlm.utils import disable_torch_init, process_images, tokenizer_image_token, KeywordsStoppingCriteria
from spatialvla.mobilevlm.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Tuple, Type

import numpy as np
import torch
import copy
from PIL import Image

from torch.utils.data import Dataset, IterableDataset

from transformers import PreTrainedTokenizerBase
from transformers import AutoTokenizer, BitsAndBytesConfig

from spatialvla.mobilevlm.utils import disable_torch_init, process_images, tokenizer_image_token, KeywordsStoppingCriteria
# from prismatic.models.backbones.llm.prompting import PromptBuilder
# from prismatic.models.backbones.vision import ImageTransform

from spatialvla.mobilevlm.constants import IGNORE_INDEX, IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from spatialvla.mobilevlm.conversation import conv_templates, SeparatorStyle
from spatialvla.mobilevlm.model.bimanual import load_twinvla
from spatialvla.datasets.rlds.utils.data_utils import tree_map
# from prismatic.vla.action_tokenizer import ActionTokenizer
from spatialvla.datasets.rlds import make_interleaved_dataset, make_single_dataset
from spatialvla.datasets.rlds.oxe import OXE_NAMED_MIXTURES, get_oxe_dataset_kwargs_and_weights
from spatialvla.datasets.rlds.utils.data_utils import NormalizationType
from transformers import PreTrainedTokenizerBase
import torch.nn as nn

2025-02-13 20:55:49.069911: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 20:55:49.101521: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-13 20:55:49.101572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-13 20:55:49.102800: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 20:55:49.108874: I tensorflow/core/platform/cpu_feature_guar

[2025-02-13 20:56:01,964] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
tokenizer, model, image_processor, _ = load_vla('checkpoints/libero_object_fm')

/home/jellyho/miniconda3/envs/mobilevlm/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/jellyho/miniconda3/envs/mobilevlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from spatialvla.mobilevlm.action_tokenizer import FASTTokenizer

fast_tokenizer = FASTTokenizer(tokenizer)

batch_transform = RLDSBatchTransform(
    tokenizer,
    image_processor,
    use_state_input=False,
    window_size=1,
    future_action_window_size=31,
    use_hz_input=True,
    action_tokenizer=fast_tokenizer
)
vla_dataset = RLDSDataset(
    data_root_dir='/home/shared/rlds_datasets',
    data_mix='libero_object_no_noops',
    batch_transform=batch_transform,
    shuffle_buffer_size=10000,
    window_size=1,
    future_action_window_size=31,
    use_state_input = False,
    quantile_norm = True
)
collator = PaddedCollatorForActionPrediction(
    tokenizer.model_max_length, 
    tokenizer.pad_token_id, 
    padding_side='right', 
    use_state_input=False,
    use_label=True,
    use_hz_input=True
)

{'name': 'libero_object_no_noops', 'data_dir': '/home/shared/rlds_datasets', 'image_obs_keys': {'primary': 'image', 'secondary': None}, 'absolute_action_mask': [False, False, False, False, False, False, False], 'action_normalization_mask': [True, True, True, True, True, True, True], 'action_proprio_normalization_type': <NormalizationType.BOUNDS_Q99: 'bounds_q99'>, 'language_key': 'language_instruction', 'standardize_fn': <function libero_dataset_transform at 0x14f2f9835750>}


2025-02-13 20:56:26.513447: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2025-02-13 20:56:27.226363: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization



######################################################################################
# Loading the following 1 datasets (incl. sampling weight):                         #
# libero_object_no_noops: ==================================================1.000000 #
######################################################################################

Threads per Dataset:  [1]
Reads per Dataset:  [1]
Constructing datasets...


2025-02-13 20:56:27.646783: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Applying frame transforms on dataset...


In [4]:
dataloader = DataLoader(
    vla_dataset,
    batch_size=4,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)

In [5]:
batch = None
for b in dataloader:
    batch = b
    break

In [7]:
batch['input_ids']

tensor([[    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -200, 29871,    13,  5618,  3158,
           881,   278, 19964,  2125,   304,  5839,   701,   278, 24841,  3623,
           625,   322,  2058,   372,   297,   278, 25972, 29973,   319,  1799,
          9047, 13566, 29901, 31716, 31688, 31702, 31697, 30264, 31596, 31728,
         31381, 31699, 31742, 31121, 31598, 31588, 31716, 31688, 31710, 31602,
         30991, 31445, 31703, 31716, 30936, 31600, 31322, 31701, 31713, 31320,
         31713, 31742, 31523, 31644, 31710, 31639, 31706, 31222, 31666, 30353,
         31604, 31416, 31604,     0],
        [    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322

In [8]:
batch['labels']

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100, 31716, 31688, 31702, 31697, 30264, 31596, 31728,
         31381, 31699, 31742, 31121, 31598, 31588, 31716, 31688, 31710, 31602,
         30991, 31445, 31703, 31716, 30936, 31600, 31322, 31701, 31713, 31320,
         31713, 31742, 31523, 31644, 31710, 31639, 31706, 31222, 31666, 30353,
         31604, 31416, 31604,  -100],
        [ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100

In [ ]:
fast = AutoProcessor.from_pretrained("physical-intelligence/fast", trust_remote_code=True)
batch = None
tokenized = []
original = []
list_mse = []
count = 0
for b in dataloader:
    batch = b
    action = batch['action']
    tokens = fast(action)
    decoded_actions = fast.decode(tokens, time_horizon=10, action_dim=7)
    mse = ((action - decoded_actions) ** 2).mean().cpu().numpy()
    for t in tokens:
        tokenized.append(len(t))
        original.append(10 * 7)
    list_mse.append(mse)
    count += 1
    print(count)
    if count == 100:
        break

In [24]:
compress_rate = np.array(tokenized).sum() / np.array(original).sum()

In [25]:
compress_rate

0.24231696428571428

In [26]:
np.array(list_mse).mean()

0.00048670708040417785

In [11]:
action = batch['action']
padded_array = np.pad(action, ((0, 0), (0, 0), (0, 32 - action.shape[-1])), mode='constant', constant_values=0)

In [64]:
action.shape

torch.Size([16, 10, 7])

In [6]:
import numpy as np
from transformers import AutoProcessor

# Load the tokenizer from the Hugging Face hub


In [9]:
len(tokens[0])

341

In [12]:
# Tokenize & decode action chunks (we use dummy data here)
tokens = fast(action)           

In [15]:
len(tokens[2])

26

In [51]:
decoded_actions = fast.decode(tokens, time_horizon=10, action_dim=32)

In [57]:
((decoded_actions[0][:, :7] - action[0].cpu().numpy())**2).mean()

0.0005500334909725041

In [89]:
((decoded_actions - batch['action'][:, :, :-1].cpu().numpy()) ** 2).mean()

ValueError: operands could not be broadcast together with shapes (16,15,7) (16,15,6) 

In [137]:
batch['action'][0][0]

tensor([ 0.0832,  0.3613,  0.4915, -0.4893,  0.4194,  0.0609,  1.0000],
       dtype=torch.float16)

In [16]:
from scipy.spatial.transform import Rotation